# Linear Regression Consulting Project

Congratulations! You've been contracted by Hyundai Heavy Industries to help them build a predictive model for some ships. [Hyundai Heavy Industries](http://www.hyundai.eu/en) is one of the world's largest ship manufacturing companies and builds cruise liners.

You've been flown to their headquarters in Ulsan, South Korea to help them give accurate estimates of how many crew members a ship will require.

They are currently building new ships for some customers and want you to create a model and use it to predict how many crew members the ships will need.

Here is what the data looks like so far:

    Description: Measurements of ship size, capacity, crew, and age for 158 cruise
    ships.


    Variables/Columns
    Ship Name     1-20
    Cruise Line   21-40
    Age (as of 2013)   46-48
    Tonnage (1000s of tons)   50-56
    passengers (100s)   58-64
    Length (100s of feet)  66-72
    Cabins  (100s)   74-80
    Passenger Density   82-88
    Crew  (100s)   90-96
    
It is saved in a csv file for you called "cruise_ship_info.csv". Your job is to create a regression model that will help predict how many crew members will be needed for future ships. The client also mentioned that they have found that particular cruise lines will differ in acceptable crew counts, so it is most likely an important feature to include in your analysis! 

Once you've created the model and tested it for a quick check on how well you can expect it to perform, make sure you take a look at why it performs so well!

In [1]:
import findspark
findspark.init('/home/ubuntu/spark-2.1.1-bin-hadoop2.7')
import pyspark
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('lRegProject').getOrCreate()

In [2]:
from pyspark.ml.regression import LinearRegression

In [3]:
data = spark.read.csv('cruise_ship_info.csv',header=True, inferSchema=True)

In [5]:
data.printSchema()

root
 |-- Ship_name: string (nullable = true)
 |-- Cruise_line: string (nullable = true)
 |-- Age: integer (nullable = true)
 |-- Tonnage: double (nullable = true)
 |-- passengers: double (nullable = true)
 |-- length: double (nullable = true)
 |-- cabins: double (nullable = true)
 |-- passenger_density: double (nullable = true)
 |-- crew: double (nullable = true)



In [6]:
data.head(1)

[Row(Ship_name='Journey', Cruise_line='Azamara', Age=6, Tonnage=30.276999999999997, passengers=6.94, length=5.94, cabins=3.55, passenger_density=42.64, crew=3.55)]

In [8]:
data.select('Cruise_line' ).show()

+-----------+
|Cruise_line|
+-----------+
|    Azamara|
|    Azamara|
|   Carnival|
|   Carnival|
|   Carnival|
|   Carnival|
|   Carnival|
|   Carnival|
|   Carnival|
|   Carnival|
|   Carnival|
|   Carnival|
|   Carnival|
|   Carnival|
|   Carnival|
|   Carnival|
|   Carnival|
|   Carnival|
|   Carnival|
|   Carnival|
+-----------+
only showing top 20 rows



In [11]:
from pyspark.ml.feature import StringIndexer
indexer = StringIndexer(inputCol='Cruise_line',outputCol='CLine_index').fit(data)

In [12]:
indexData=indexer.transform(data)

In [13]:
indexData.printSchema()

root
 |-- Ship_name: string (nullable = true)
 |-- Cruise_line: string (nullable = true)
 |-- Age: integer (nullable = true)
 |-- Tonnage: double (nullable = true)
 |-- passengers: double (nullable = true)
 |-- length: double (nullable = true)
 |-- cabins: double (nullable = true)
 |-- passenger_density: double (nullable = true)
 |-- crew: double (nullable = true)
 |-- CLine_index: double (nullable = true)



In [16]:
indexData.columns

['Ship_name',
 'Cruise_line',
 'Age',
 'Tonnage',
 'passengers',
 'length',
 'cabins',
 'passenger_density',
 'crew',
 'CLine_index']

In [17]:
from pyspark.ml.linalg import Vector
from pyspark.ml.feature import VectorAssembler

In [20]:
assembler = VectorAssembler(inputCols=['Age', 'Tonnage', 'passengers', 'length', 'cabins', 'passenger_density', 'CLine_index'],outputCol='features')

In [21]:
output=assembler.transform(indexData)

In [22]:
output.printSchema()

root
 |-- Ship_name: string (nullable = true)
 |-- Cruise_line: string (nullable = true)
 |-- Age: integer (nullable = true)
 |-- Tonnage: double (nullable = true)
 |-- passengers: double (nullable = true)
 |-- length: double (nullable = true)
 |-- cabins: double (nullable = true)
 |-- passenger_density: double (nullable = true)
 |-- crew: double (nullable = true)
 |-- CLine_index: double (nullable = true)
 |-- features: vector (nullable = true)



In [23]:
final_data =output.select('features','crew')

In [24]:
final_data.show()

+--------------------+----+
|            features|crew|
+--------------------+----+
|[6.0,30.276999999...|3.55|
|[6.0,30.276999999...|3.55|
|[26.0,47.262,14.8...| 6.7|
|[11.0,110.0,29.74...|19.1|
|[17.0,101.353,26....|10.0|
|[22.0,70.367,20.5...| 9.2|
|[15.0,70.367,20.5...| 9.2|
|[23.0,70.367,20.5...| 9.2|
|[19.0,70.367,20.5...| 9.2|
|[6.0,110.23899999...|11.5|
|[10.0,110.0,29.74...|11.6|
|[28.0,46.052,14.5...| 6.6|
|[18.0,70.367,20.5...| 9.2|
|[17.0,70.367,20.5...| 9.2|
|[11.0,86.0,21.24,...| 9.3|
|[8.0,110.0,29.74,...|11.6|
|[9.0,88.5,21.24,9...|10.3|
|[15.0,70.367,20.5...| 9.2|
|[12.0,88.5,21.24,...| 9.3|
|[20.0,70.367,20.5...| 9.2|
+--------------------+----+
only showing top 20 rows



In [25]:
train_data, test_data = final_data.randomSplit([0.7,0.3])

In [26]:
lr= LinearRegression(labelCol='crew',predictionCol='predictionCrew',featuresCol='features')

In [27]:
lr_model = lr.fit(train_data)

In [28]:
lr_testresult=lr_model.evaluate(test_data)

In [29]:
lr_testresult.rootMeanSquaredError

1.0459556634414093

In [30]:
lr_testresult.r2

0.8692113712698959

In [31]:
unlabeledData=test_data.select('features')

In [32]:
prediction = lr_model.transform(unlabeledData)

In [33]:
prediction.show()

+--------------------+------------------+
|            features|    predictionCrew|
+--------------------+------------------+
|[5.0,86.0,21.04,9...| 9.292170951350496|
|[5.0,115.0,35.74,...|11.820412940676027|
|[5.0,122.0,28.5,1...| 7.019236767514627|
|[6.0,93.0,23.94,9...|10.474177236317393|
|[6.0,113.0,37.82,...|11.755068438910806|
|[6.0,158.0,43.7,1...|14.109895745045788|
|[8.0,91.0,22.44,9...|10.033652624589548|
|[9.0,81.0,21.44,9...| 9.454736157208382|
|[9.0,85.0,19.68,9...| 9.259851711899966|
|[10.0,58.825,15.6...|7.2390582073682115|
|[10.0,81.76899999...| 8.820779896455457|
|[10.0,86.0,21.14,...|  9.61014417141985|
|[11.0,90.09,25.01...| 9.056954342968774|
|[12.0,25.0,3.88,5...|3.0862438758207213|
|[12.0,50.0,7.0,7....| 4.662318345127309|
|[12.0,77.104,20.0...| 8.707645711829164|
|[13.0,25.0,3.82,5...|3.0778260626467926|
|[13.0,61.0,13.8,7...|  6.55034319576198|
|[13.0,76.0,18.74,...| 8.654123888035043|
|[14.0,33.0,4.9,5....| 3.226706765085475|
+--------------------+------------